In [1]:
import numpy as np

In [2]:
class SCFSolver:
    '''
    Reference: A.Szabo, N.S.Ostlund
    https://www.amazon.co.jp/-/en/Attila-Szabo/dp/4130621114/ref=pd_lpo_14_t_0/357-6877730-3564413?_encoding=UTF8&pd_rd_i=4130621114&pd_rd_r=21208b56-0531-4686-a004-b21b0d615dbe&pd_rd_w=r2Xpu&pd_rd_wg=oGSC6&pf_rd_p=cb2cef9d-b0a3-4b58-a575-45abfc5e07e8&pf_rd_r=3C5KDGNZX2WMMXDKWT0T&psc=1&refRID=3C5KDGNZX2WMMXDKWT0T
    '''
    def __init__(self, Hcore: np.array, S: np.array, two_elec_integral: np.array, num_elec: int, num_AO: int):
        '''
        parameter Hcore: Hamiltonial Matrix
        parameter S: Overlap Integral Matrix
        parameter two_elec_integral: 2 Electron Integral Matrix
        parameter num_elec: Number of Electrons
        parameter num_AO: Number of Atomic Orbitals
        '''
        self.Hcore = Hcore
        self.S = S
        self.two_elec_integral = two_elec_integral
        self.num_elec = num_elec
        self.num_AO = num_AO
        assert self.Hcore.shape == self.S.shape

    def compute_transformation_matrix(self, mode='canonical'):
        '''
        parameter mode should be 'canonical' or 'symmetric'
        return: regular matrix
        '''
        X = np.zeros(self.S.shape)
        
        #固有値の配列、固有ベクトルをそれぞれ返す
        e, e_v = np.linalg.eigh(self.S)
        if mode=='canonical':
            #正準直交化
            for i in range(self.num_AO):
                for j in range(self.num_AO):
                    X[i,j] = e_v[i,j] / np.sqrt(e[j])
        elif mode=='symmetric':
            #対称直交化
            regular_matrix = np.zeros(self.S.shape)
            for i in range(len(e)):
                regular_matrix[i,i] = 1/np.sqrt(e[i])
            X = e_v.dot(regular_matrix.dot(np.conjugate(e_v)))
        return X
    
    def update_P_matrix(self, P, C):
        '''
        parameter P: density matrix
        parameter C_dash: X*C(C: regular matrix computed from Fock matrix)
        return: updated density matrix
        '''
        new_P = np.zeros(P.shape)
        ### 占有軌道の数
        num_occupied_orbital = self.num_elec // 2 
        for i in range(self.num_AO):
            for j in range(self.num_AO):
                tmp = 0
                for k in range(num_occupied_orbital):
                    tmp += 2*(C[i,k]*C[j,k])
                new_P[i,j] = tmp
        return new_P

    def compute_term_2_elec_integral(self, P: np.array):
        '''
        2電子積分(self.two_elec_integral)がエルミート行列であることを確認
        '''
        new_G = np.zeros((self.num_AO,self.num_AO))

        for i in range(self.num_AO):
            for j in range(self.num_AO):
                for k in range(self.num_AO):
                    for l in range(self.num_AO):
                        J = self.two_elec_integral[i][j][k][l]
                        K = 0.5 * self.two_elec_integral[i][l][k][j]
                        new_G[i,j] += P[k,l]*(J-K)
        return new_G

    def judge_scf_end(self, new_P: np.array, P: np.array):
        '''
        parameter P: density matrix
        return: T/F (if True: Finish SCF loops)
        '''
        THRESHOLD=1e-4
        diff_val = 0.5 * ((new_P-P)**2).sum()
        do_finish = False
        if diff_val < THRESHOLD:
            do_finish = True
        return do_finish

    def scf(self):
        X = self.compute_transformation_matrix()
        X_adj = np.matrix.getH(X) #共役転置行列を得る
        P = np.zeros(self.S.shape)
        MAX_ITER = 100

        for n_iter in range(MAX_ITER):
            print('-'*50)
            print(f"ITERATION={n_iter+1}/{MAX_ITER}")
            G = self.compute_term_2_elec_integral(P)
            Fock = self.Hcore + G
            Fock_dash = X_adj.dot(Fock.dot(X))
            eigen_val, C_dash = np.linalg.eigh(Fock_dash)
            C = X.dot(C_dash)
            new_P = self.update_P_matrix(P, C)

            #参考: https://qiita.com/hatori_hoku/items/867fa793488ebe1d2beb
            energy = (new_P * (self.Hcore + Fock)).sum() * 0.5 
            print(f'Electronic Energy = {energy} hartrees')
            do_finish = self.judge_scf_end(new_P, P)
            if do_finish:
                print('\nFinish SCF Calculation\n')
                print('\nComputed Density Matrix')
                print(new_P)
                print('-'*50)
                break
            P = new_P
            print('-'*50, '\n'*3)

In [3]:
num_AO = 2
Hcore = np.zeros((num_AO, num_AO))
Hcore[0,0] = Hcore[1,1] = (0.7600 + (-1.2266) + (-0.6538))
Hcore[1,0] = Hcore[0,1] = 0.2365 + (-0.5974) + (-0.5974)

S = np.zeros((num_AO, num_AO))
S[0,0] = S[1,1] = 1
S[0,1] = S[1,0] = 0.6593

two_elec_integral_matrix = np.zeros((num_AO, num_AO, num_AO, num_AO))
two_elec_integral_matrix[0][0][0][0] = two_elec_integral_matrix[1][1][1][1] = 0.7746
two_elec_integral_matrix[1][1][0][0] = two_elec_integral_matrix[0][0][1][1] = 0.5697
two_elec_integral_matrix[1][0][0][0] = two_elec_integral_matrix[0][0][0][1] = two_elec_integral_matrix[0][0][1][0] = two_elec_integral_matrix[0][1][0][0] = 0.4441
two_elec_integral_matrix[1][1][1][0] = two_elec_integral_matrix[0][1][1][1] = two_elec_integral_matrix[1][1][0][1] = two_elec_integral_matrix[1][0][1][1] = 0.4441
two_elec_integral_matrix[1][0][1][0] = two_elec_integral_matrix[0][1][0][1] = two_elec_integral_matrix[0][1][1][0] = two_elec_integral_matrix[1][0][0][1] = 0.2970

In [4]:
solver = SCFSolver(Hcore=Hcore, S=S, two_elec_integral=two_elec_integral_matrix, num_elec=2, num_AO=2)

In [5]:
solver.scf()

--------------------------------------------------
ITERATION=1/100
Electronic Energy = -2.5055143735310073 hartrees
-------------------------------------------------- 



--------------------------------------------------
ITERATION=2/100
Electronic Energy = -1.830918121848344 hartrees

Finish SCF Calculation


Computed Density Matrix
[[0.60266377 0.60266377]
 [0.60266377 0.60266377]]
--------------------------------------------------


上のComputed Density Matrixの値は1/(1+S)の値に等しい(Sは電子1,2の重なり積分の値で0.6593)

In [6]:
'''
A.Szabo, N.S.Ostlund '新しい量子化学上巻', 巻末より引用
'''
Hcore = np.zeros((num_AO, num_AO))
T11 = 2.164313
T12 = 0.167013
T21 = T12
T22 = 0.760033

V11A = -4.139827
V12A = -1.102912
V21A = V12A
V22A = -1.265246

V11B = -0.677230
V12B = -0.411305
V21B = V12B
V22B = -1.226615

Hcore[0,0] = T11 + V11A + V11B 
Hcore[0,1] = Hcore[1,0] = T12 + V12A + V12B
Hcore[1,1] = T22 + V22A + V22B

S = np.zeros((num_AO, num_AO))  # Overlap matrix
S[0,0] = S[1,1] = 1
S[0,1] = S[1,0] = 0.45077

two_elec_integral_matrix = np.zeros((num_AO, num_AO, num_AO, num_AO))

V1111 = 1.307152 
V2111 = 0.437279
V2121 = 0.177267 
V2211 = 0.605703 
V2221 = 0.311795 
V2222 = 0.774608

two_elec_integral_matrix[0][0][0][0] = V1111
two_elec_integral_matrix[0][0][1][1] = two_elec_integral_matrix[1][1][0][0] = V2211
two_elec_integral_matrix[0][0][0][1] = two_elec_integral_matrix[0][0][1][0] = two_elec_integral_matrix[0][1][0][0] = two_elec_integral_matrix[1][0][0][0] = V2111
two_elec_integral_matrix[0][1][0][1] = two_elec_integral_matrix[0][1][1][0] = two_elec_integral_matrix[1][0][0][1] = two_elec_integral_matrix[1][0][1][0] = V2121
two_elec_integral_matrix[0][1][1][1] = two_elec_integral_matrix[1][0][1][1] = two_elec_integral_matrix[1][1][0][1] = two_elec_integral_matrix[1][1][1][0] = V2221
two_elec_integral_matrix[1][1][1][1] = V2222

In [7]:
solver = SCFSolver(Hcore=Hcore, S=S, two_elec_integral=two_elec_integral_matrix, num_elec=2, num_AO=2)

In [8]:
solver.scf()

--------------------------------------------------
ITERATION=1/100
Electronic Energy = -5.348170796777179 hartrees
-------------------------------------------------- 



--------------------------------------------------
ITERATION=2/100
Electronic Energy = -4.143489094522493 hartrees
-------------------------------------------------- 



--------------------------------------------------
ITERATION=3/100
Electronic Energy = -4.2189015868815645 hartrees
-------------------------------------------------- 



--------------------------------------------------
ITERATION=4/100
Electronic Energy = -4.226857722919284 hartrees

Finish SCF Calculation


Computed Density Matrix
[[1.28642446 0.54003971]
 [0.54003971 0.22670814]]
--------------------------------------------------
